<a href="https://colab.research.google.com/github/hyesukim1/Video-game-sales-project/blob/main/Video_game_sales_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone [github link]

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/datasets/vgames2.csv')
#df.drop(['Unnamed: 0'], axis = 1, inplace = True)#unnamed 컬럼 제거

In [ ]:
#unnamed 컬럼 제거
df.drop(['Unnamed: 0'], axis = 1, inplace = True)

#year데이터 2글자를 4글자로 수정
def year(x):
   if x <= 20:
     x += 2000
     return x
   elif x > 20 and x <= 100:
     x += 1900
     return x
   else:
     return x

df['Year'] = df['Year'].apply(year)


In [ ]:
def M(string):
  return str(string.replace('M',''))

df['NA_Sales'] = df['NA_Sales'].apply(M)
df['EU_Sales'] = df['EU_Sales'].apply(M)
df['JP_Sales'] = df['JP_Sales'].apply(M)
df['Other_Sales'] = df['Other_Sales'].apply(M)

In [ ]:
df = df[~df['NA_Sales'].str.contains("K")]
df = df[~df['EU_Sales'].str.contains("K")]
df = df[~df['JP_Sales'].str.contains("K")]
df = df[~df['Other_Sales'].str.contains("K")]

In [ ]:
import pandas as pd
df['NA_Sales'] = df['NA_Sales'].apply(pd.to_numeric)
df['EU_Sales'] = df['EU_Sales'].apply(pd.to_numeric)
df['JP_Sales'] = df['JP_Sales'].apply(pd.to_numeric)
df['Other_Sales'] = df['Other_Sales'].apply(pd.to_numeric)

df.head()

In [ ]:
df['Global_Sales'] = df[['NA_Sales','EU_Sales','JP_Sales','Other_Sales']].sum(axis=1)
df

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import statsmodels.stats.multicomp as mc
import matplotlib.pyplot as plt
import seaborn as sns

#Q1.지역에 따라 선호하는 게임 장르 확인, 지역별 팔린 게임의 평균은 같은가?
df1 = df.iloc[:,[3,5,6,7,8]]
df1

In [ ]:
#아노바 테스트 진행: P값이 0.005이하로 귀무가설을 기각하고 대립가설을 채택

import scipy.stats as stats
import pandas as pd
import urllib
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import seaborn as sns
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
plt.rc('font', family='NanumBarunGothic') 
plt.rc('axes', unicode_minus=False)

df_melt = df.iloc[:,[5,6,7,8]]

df_melt1 = pd.melt(df_melt.reset_index(), id_vars=['index'], value_vars=['NA_Sales', 'EU_Sales', 'JP_Sales','Other_Sales'])
df_melt1

F_statistic, pVal = stats.f_oneway(df['NA_Sales'],df['EU_Sales'],df['JP_Sales'],df['Other_Sales'])

print('아노바테스트 : F={0:.1f}, p={1:.10f}'.format(F_statistic, pVal))

In [ ]:
#북미지역 판매량 순으로 장르 확인

df_Nas = pd.pivot_table(df,
                     index='Genre', 
                     values='NA_Sales', 
                     aggfunc=np.sum)
df_Nas1 = df_Nas.sort_values(by='NA_Sales',ascending=False)
df_Nas1.plot.bar(color='green', alpha=0.5)
plt.show()

In [ ]:
#유럽지역 판매량 순으로 장르 확인
df_Eus = pd.pivot_table(df,
                     index='Genre', 
                     values='EU_Sales', 
                     aggfunc=np.sum)
df_Eus1 = df_Eus.sort_values(by='EU_Sales',ascending=False)
df_Eus1.plot.bar(color='green', alpha=0.5)
plt.show()

In [ ]:
#일본지역 판매량 순으로 장르 확인
df_Jps = pd.pivot_table(df,
                     index='Genre', 
                     values='JP_Sales', 
                     aggfunc=np.sum)
df_Jps1 = df_Jps.sort_values(by='JP_Sales',ascending=False)
df_Jps1.plot.bar(color='green', alpha=0.5)
plt.show()

In [ ]:
#기타지역 판매량 순으로 장르 확인
df_other = pd.pivot_table(df,
                     index='Genre', 
                     values='Other_Sales', 
                     aggfunc=np.sum)
df_other1 = df_other.sort_values(by='Other_Sales',ascending=False)
df_other1.plot.bar(color='green', alpha=0.5)
plt.show()

___
___

In [ ]:
#연도별 각 장르의 판매합계 확인
import matplotlib.pyplot as plt
%matplotlib inline

df_aff = df.groupby(by=['Year', 'Genre']).Global_Sales.sum().reset_index()
pivot_df = pd.pivot_table(df_aff,
                          index= 'Year', 
                          columns='Genre', 
                          values='Global_Sales', 
                          aggfunc='sum')
pivot_df.fillna(0)


pivot_df.plot.bar(stacked=True, figsize=(10,7))

In [ ]:
#년도별 세계 게임 구매량 추이 
df_aff = df.groupby(by=['Year', 'Genre']).Global_Sales.sum().reset_index()
sns.relplot(kind="line", data=df_aff, x="Year", y="Global_Sales", hue="Genre")

In [ ]:
#장르별로 확인
sns.relplot(
    kind="line", data=df_aff, x="Year", y="Global_Sales", hue="Genre", 
    col="Genre", col_wrap=4)

In [ ]:
# 년도별 최다 판매량 기록한 장르 데이터프레임
year_max_sales = df.groupby(['Year', 'Genre'])['Global_Sales'].sum().reset_index()
condition = year_max_sales['Global_Sales'] == year_max_sales.groupby(['Year'])['Global_Sales'].transform(max)
year_max_sales = year_max_sales[condition]
year_max_sales.columns = ['년도', '장르', '전세계 판매량']

# 장르 데이터 저장
genre = year_max_sales['장르'].values

# 그래프 그리기
plt.figure(figsize=(28,10))
ax = sns.barplot(x='년도', y='전세계 판매량', data=year_max_sales)
idx = 0
for value in year_max_sales['전세계 판매량']:
    ax.text(x=idx, y=value + 2, s=str(genre[idx] + '---' + ' ' + str(round(value, 2))),
            color='black', size=14, rotation=90, ha='center')
    idx += 1
ax.set_title('Year per Global Sales', y=1.06, fontsize=28, loc='left')
plt.xticks(rotation=90, fontsize=12)
plt.yticks(fontsize=12)
plt.xlabel('Year', fontsize=20)
plt.ylabel('Global Sales', fontsize=20)
plt.show()

In [ ]:
#북미 장르별 최고 판매량

region_top = df1.groupby('Genre').sum().reset_index()
region_top

#Region_top3 데이터
NA_top = region_top.loc[:, ['Genre','NA_Sales']]
NA_top1 = NA_top.sort_values(by=['NA_Sales'],ascending=False).head(3)
NA_top1
sns.barplot(
    data= NA_top1,
    x= "Genre",
    y= "NA_Sales"
)
plt.show()


In [ ]:
#유럽 장르별 최고 판매량

EU_top = region_top.loc[:, ['Genre','EU_Sales']]
EU_top1 = EU_top.sort_values(by=['EU_Sales'],ascending=False).head(3)

sns.barplot(
    data= EU_top1,
    x= "Genre",
    y= "EU_Sales"
)
plt.show()

In [ ]:
#일본 장르별 최고 판매량

JP_top = region_top.loc[:, ['Genre','JP_Sales']]
JP_top1 = JP_top.sort_values(by=['JP_Sales'],ascending=False).head(3)

sns.barplot(
    data= JP_top1,
    x= "Genre",
    y= "JP_Sales"
)
plt.show()

In [ ]:
#가타 장르별 최고 판매량

Other_top = region_top.loc[:, ['Genre','Other_Sales']]
Other_top1 = Other_top.sort_values(by=['Other_Sales'],ascending=False).head(3)

sns.barplot(
    data= Other_top1,
    x= "Genre",
    y= "Other_Sales"
)
plt.show()

In [ ]:
#플랫폼 지역출고량 top3
df2 = df.iloc[:,[1,5,6,7,8]]
df2

platform_top = df2.groupby('Platform').sum().reset_index()
platform_top.head()

In [ ]:
#Platform_top3 데이터_NA
P_NA_top = platform_top.loc[:, ['Platform','NA_Sales']]
P_NA_top1 = P_NA_top.sort_values(by=['NA_Sales'],ascending=False).head(3)

sns.barplot(
    data= P_NA_top1,
    x= "Platform",
    y= "NA_Sales"
)
plt.show()

In [ ]:
#Platform_top3 데이터_EU
P_EU_top = platform_top.loc[:, ['Platform','EU_Sales']]
P_EU_top1 = P_EU_top.sort_values(by=['EU_Sales'],ascending=False).head(3)

sns.barplot(
    data= P_EU_top1,
    x= "Platform",
    y= "EU_Sales"
)
plt.show()

In [ ]:
#Platform_top3 데이터_JP
P_JP_top = platform_top.loc[:, ['Platform','JP_Sales']]
P_JP_top1 = P_JP_top.sort_values(by=['JP_Sales'],ascending=False).head(3)

sns.barplot(
    data= P_JP_top1,
    x= "Platform",
    y= "JP_Sales"
)
plt.show()

In [ ]:
#Platform_top3 데이터_other
P_Other_top = platform_top.loc[:, ['Platform','Other_Sales']]
P_Other_top1 = P_Other_top.sort_values(by=['Other_Sales'],ascending=False).head(3)

sns.barplot(
    data= P_Other_top1,
    x= "Platform",
    y= "Other_Sales"
)
plt.show()

In [ ]:
#퍼블리셔(회사)별 지역 출고량 top3

df3 = df.iloc[:,[4,5,6,7,8]]
df3

In [ ]:
publisher_top = df3.groupby('Publisher').sum().reset_index()
publisher_top.head()

Pu_NA_top = publisher_top.loc[:, ['Publisher','NA_Sales']]
Pu_NA_top1 = Pu_NA_top.sort_values(by=['NA_Sales'],ascending=False).head(3)

sns.barplot(
    data= Pu_NA_top1,
    x= "Publisher",
    y= "NA_Sales"
)
plt.show()

In [ ]:
Pu_EU_top = publisher_top.loc[:, ['Publisher','EU_Sales']]
Pu_EU_top1 = Pu_EU_top.sort_values(by=['EU_Sales'],ascending=False).head(3)

sns.barplot(
    data= Pu_EU_top1,
    x= "Publisher",
    y= "EU_Sales"
)
plt.show()

In [ ]:
Pu_JP_top = publisher_top.loc[:, ['Publisher','JP_Sales']]
Pu_JP_top1 = Pu_JP_top.sort_values(by=['JP_Sales'],ascending=False).head(3)

sns.barplot(
    data= Pu_JP_top1,
    x= "Publisher",
    y= "JP_Sales"
)
plt.show()

In [ ]:
Pu_Other_top = publisher_top.loc[:, ['Publisher','Other_Sales']]
Pu_Other_top1 = Pu_Other_top.sort_values(by=['Other_Sales'],ascending=False).head(3)

sns.barplot(
    data= Pu_Other_top1,
    x= "Publisher",
    y= "Other_Sales"
)
plt.show()

In [ ]:
#출고량이 높은 게임 시각화
import matplotlib.font_manager as fm

#게임별 글로벌 판매량 상위 10개 데이터 추출
df_game_name = df.loc[df.Global_Sales.sort_values(ascending=False).index].reset_index().head(10)
df_game_name #연도가 섞여있음(날짜 기준 넣고 수정 필요)


df_game_name_glo = df_game_name.iloc[:10,[1,10]]
df_game_name_glo

df_game_name_glo.plot.barh(x='Name', y='Global_Sales',  cmap='Pastel1')
plt.xlabel('Total Sales')
plt.ylabel('Game Name')
plt.show()#Wii sports>Super Mario Bros > Mario Kart Wii 순으로 높은것을 확인

In [ ]:
#파이차트 데이터 준비
labels = ['Wii Sports', 'Super Mario Bros', 'Mario Kart Will', 'Wii Sports Resort', 'Pokemon Red/Pokemon Blue'] # 라벨설정
labels
frequency = [82,40,35,33,31] # 빈도 설정
 
fig = plt.figure(figsize=(8,8)) # 캔버스 생성
fig.set_facecolor('white') # 캔버스 배경색을 하얀색으로 설정

explode = [0.1,0,0,0,0]
plt.pie(frequency, # 파이차트 출력
       explode = explode,
       labels=labels,
       shadow=True,
       startangle=90, #시작점을 90도(degree)로 지정
       textprops={'fontsize': 14},
       counterclock=False, # 시계 방향으로 그린다.
       autopct=lambda p : '{:.2f}%'.format(p), # 퍼센티지 출력
       colors = ['palevioletred','lightpink','pink','lavenderblush','snow']
       )

plt.title('Total Top 5 sales for game', fontsize=20)
plt.show()

In [ ]:
#2017년 기준으로 최신 5년내에 글로벌 판매량 상위 10개 게임 데이터 추출

df_game_name_glo_Y = df[df.Year >= 2013].sort_values('Global_Sales', ascending=False).head(10)
df_game_name_glo_Y = df_game_name_glo_Y.reset_index(drop=True)
df_game_name_glo_Y

df_game_name_glo_Y_cut= df_game_name_glo_Y.iloc[:10,[0,9]]
df_game_name_glo_Y_cut

df_game_name_glo_Y_cut.plot.barh(x='Name', y='Global_Sales', color='lightsteelblue')
plt.xlabel('Total Sales')
plt.ylabel('Game Name')
plt.show()

In [ ]:
labels = ['Grand Theft Auto V(PS3)', 'Grand Theft Auto V(X360)', 'Call of Duty: Black Ops 3', 'Grand Theft Auto V(PS4)', 'Pokemon Omega Ruby/Pokemon Alpha Sapphire'] # 라벨
labels
colors = ['royalblue','cornflowerblue','lighskyblue','lightcyan','aliceblue']
colors = colors
frequency = [21,16,14,11,11] # 빈도
 
fig = plt.figure(figsize=(8,8)) # 캔버스 생성
fig.set_facecolor('white') # 캔버스 배경색을 하얀색으로 설정

explode = [0.1,0,0,0,0]
plt.pie(frequency, # 파이차트 출력
       explode = explode,
       labels=labels,
       shadow=True,
       startangle=90, # 시작점을 90도(degree)로 지정
       textprops={'fontsize': 14},
       counterclock=False, # 시계 방향으로 그린다.
       autopct=lambda p : '{:.2f}%'.format(p), # 퍼센티지 출력
       colors = ['royalblue','cornflowerblue','lightskyblue','aliceblue','ghostwhite']
       )

plt.title('Top 5 sales for game_recent 5years', fontsize=20)
plt.show()


